In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

In [0]:
ott_df=spark.read.format('delta').option('header','True').load('abfss://bronze@storageottanalytics.dfs.core.windows.net/ott_titles')

In [0]:
ott_df=ott_df.drop('_rescued_data')

ott_df=ott_df.fillna({'duration_minutes':'0','duration_seasons':'1'})  ##Fill NULLs with a default value
ott_df=ott_df.filter(col('duration_minutes').rlike('^[0-9]+$'))  ##Filter records which are invalid

ott_df=ott_df.withColumn('duration_minutes',col('duration_minutes').cast('int')).withColumn('duration_seasons',col('duration_seasons').cast('int')) ##Convert columns to int 
ott_df=ott_df.withColumn('release_year',col('release_year').cast('int'))

ott_df=ott_df.withColumn('short_title',split(col('title'),':')[0])  ##Transformation on Title Column - to bring in the short title

ott_df=ott_df.withColumn('rating',split(col('rating'),'-')[0]) ##Transformation on Rating column


In [0]:
ott_df=ott_df.withColumn('typeFlag',when(col('type')=='Movie',1).when(col('type')=='TV Show',2).otherwise(0))

ott_df=ott_df.withColumn('duration_rank',dense_rank().over(Window.orderBy(col('duration_minutes').desc())))


In [0]:
ott_df.display()

In [0]:
ott_df.write.format('delta').mode('append').option('path','abfss://silver@storageottanalytics.dfs.core.windows.net/ott_titles')\
    .option('mergeSchema','true').save()